In [7]:
!pip install 'numpy<2'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 83.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
ERROR: Could not install packages due to an OSError: [Errno 16] Device or resource busy: '.nfs4b7b08b22a8b7e1f00000006'


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import json
import imagesize
import torch
import glob
import random
import cv2
import numpy as np
from torch.utils.data import Dataset, RandomSampler, DataLoader, Sampler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig
import pickle
from collections import defaultdict
# In[2]:


from peft import LoraConfig, get_peft_model, TaskType
import transformers
# from transformers import AdamW
from accelerate import Accelerator
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import bitsandbytes as bnb
from transformers import Trainer, TrainingArguments

## Microsoft Phi_4_instruct

In [3]:
## tokenizer

local_cache = './phi_4_model/'
name = "microsoft/Phi-4-mini-instruct"
# local_cache = './mistral_models'
# name = "mistralai/Mistral-7B-Instruct-v0.3"

#http://localhost:8889/edit/mnt/efs/RaghavWork/VirtualENV/1shot/lib/python3.10/site-packages/transformers/models/mistral/modeling_mistral.py
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(name, cache_dir=local_cache)
tokenizer.pad_token = tokenizer.eos_token  # For causal LM

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    name,
    cache_dir=local_cache,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
messages = [
    {"role": "system", "content": "You are a helpful AI assistant.Your task is to extract key-value information from the document. Refer to the example input and output and then return the output for the new input. Don't produce any note or other informtion at end"},
    {"role": "user", "content": "Input: Southampton , United Kingdom 9403.60 7,342.50 kg AMS Ref . No. 125 CARTONS TCLU8883933 PO FLEX - 2669545 FLXT - 2669545-3215367 Flexport Carrier Booking Operations FLEXPORT INTERNATIONAL B.V. VIJZELSTRAAT 68 , 1017 HL AMSTERDAM NETHERLANDS ARRIVALNOTICES_EU@FLEXPORT.COM Flexport's terms and conditions of service pursuant to this bill of lading or when acting in the capacity of a non - - operating common Loaded on Board carrier are incorporated by reference and can be found at https://www.flexport.com/terms-and-conditions/ . Additionally Additionally for all Sea Waybills Waybills the CMI Uniform Rules for Sea Waybills shall also apply ( and shall govern in accordance with Flexport's terms and conditions conditions ) pursuant to Jun 04 , 2024 which Shipper irrevocably relinquishes its right to control over the shipment by authorizing Flexport to proceed with the issuance of a Sea Waybill . International LLC By By Flexport As Carrier FMC FMC OTI No. 025219NF X Jun 04 , 2024 COPY BILL OF LADING FOR PORT - TO - PORT OR COMBINED TRANSPORT , , , . Pre - carriage By flexport . Agent Reference Number Point of Origin / FTZ Number Containerized FCL Marks and Numbers No. of Containers Description of Packages and Goods Said to Contain - Shipper's Load Stow & Count ( SLAC ) 1 x 40 ft HC Dry Container SLAC 125 CARTONS Total PO # 4127786 HS CODE : 9403.60 , 45 CTNS / 2902.5 KGS / 28.08 CBM HLM002 HELMER BLACKENED OAK LARGE TV UNIT . , 80 CTNS / 4440 KGS / 38.8 CBM SLAC 7,342.50 kg 66.880 cbm Declared Value Containers Carrier's receipt . Total number of containers or packages received by carrier : 1 container ( s ) vessel , , , : - Freight Charges ( and description ) Freight Collect This bill of lading is subject to Flexport's standard terms and conditions of carriage , which appear hereof as page 2 Page 1 of 2 Number Bill of Lading ONEYHKGE93028600 Southampton , United Kingdom Vessel & Voyage SAME AS CNEE CNCS20288 Measurement Notify Party HOARE GROUP LIMITED 3000A Parkway Parkway Whiteley Whiteley Hampshire , PO15 7FX 7FX United Kingdom FLXT - 00002669545B Seal shipped to Consignee / FLXT00002669545B FURNITURE HLM001 HELMER BLACKENED OAK 4 DOORS SIDEBOARD , Container China , Yantian Shipped From Exporter / Port of Loading HS CODE : Special Instructions Booking Number NRA Number Place of Pickup Forwarding Agent Destination Agent B / L Issue Date 028W - AL JMELIYAH Places of Delivery Gross Weight 4127786 MBL Ref Ref No. Port of Unloading CY - CY 66.880 cbm Ltd Direct Buy It TRIDENT BUSINESS PARK / NEPTUNE WAY / LEEDS RD , , HUDDERSFIELD , HD2 1UA , United Kingdom"},
    {"role": "assistant", "content": "Output: {'UNLADING_PORT': 'Southampton , United Kingdom', 'HTS_NUMBER': '9403.60', 'GROSS_WEIGHT': '7,342.50 kg', 'NUMBER_OF_PACKAGES': '125 CARTONS', 'CONTAINER_NUMBER': 'TCLU8883933', 'MASTER_BILL_OF_LADING': 'ONEYHKGE93028600', 'PLACE_OF_DELIVERY': 'Southampton , United Kingdom', 'NOTIFY_PARTY': 'SAME AS CNEE', 'SEAL_NUMBER': 'CNCS20288', 'SHIPPER': 'HOARE GROUP LIMITED 3000A Parkway Parkway Whiteley Whiteley Hampshire , PO15 7FX 7FX United Kingdom', 'BILL_OF_LADING': 'FLXT - 00002669545B', 'AMS_BL': 'FLXT00002669545B', 'PRODUCT_DESCRIPTION': 'FURNITURE HLM001 HELMER BLACKENED OAK 4 DOORS SIDEBOARD ,', 'LADING_PORT': 'China , Yantian', 'VESSEL_VOYAGE': '028W - AL JMELIYAH', 'PO_NUMBER': '4127786', 'CONTRACT_OF_CARRIAGE': 'CY - CY', 'VOLUME': 'cbm 66.880', 'CONSIGNEE': 'Ltd Direct Buy It TRIDENT BUSINESS PARK / NEPTUNE WAY / LEEDS RD , , HUDDERSFIELD , HD2 1UA , United Kingdom'}"},
    {"role": "user", "content": "New Input: Southampton , United Kingdom 5,540.00 kg 11,056.00 kg AMS Ref . No. 1854 CARTONS YMMU6326692 PO PO FLEXPORT INTERNATIONAL B.V. VIJZELSTRAAT 68 , 1017 HL AMSTERDAM NETHERLANDS ARRIVALNOTICES_EU@FLEXPORT.COM CML Grandsight Supply Chain Management Co. , Ltd . ( PREPAYMENT PREPAYMENT FLXT - 2617825-3170083 FLEX - 2617825 Flexport's terms and conditions of service pursuant to this bill of lading or when acting in the capacity of a non - - operating common Loaded on Board carrier are incorporated by reference and can be found at https://www.flexport.com/terms-and-conditions/ . Additionally Additionally for all Sea Waybills , the CMI Uniform Rules for Sea Waybills shall also apply ( ( shall govern in accordance with Flexport's terms and conditions ) ) pursuant to Apr 22 22 2024 which Shipper irrevocably relinquishes its right to control over the shipment by authorizing Flexport to proceed with the issuance of a Sea Waybill . By By Flexport International LLC As Carrier FMC - OTI No. 025219NF This bill of lading is subject to Flexport's standard terms and conditions of carriage , which appear hereof as page 2 Page 1 of 3 COPY BILL OF LADING FOR PORT - TO - PORT OR COMBINED TRANSPORT flexport . Place of Pickup B / L Issue Date Apr 25 , 2024 Pre - carriage By Neptune Leeds / Leeds ) Agent Reference Number Point of Origin / FTZ Number Marks and Numbers No. of Containers Said to Contain - Shipper's Load Stow & Count ( SLAC ) 1 x 40 ft HC Dry Container SLAC 197 CARTONS N / M 1 x 40 ft HC Dry N / M See attached pages for more container information Total Container SLAC 366 CARTONS SLAC 47,890.00 kg 357.950 cbm Declared Value Containers Carrier's receipt . Total number of containers or packages received by carrier : 5 container ( s ) vessel , and , : Freight Charges ( and description ) Freight Collect , Number Bill of Lading YMJAN235174064 Southampton , United Kingdom Vessel & Voyage Ltd. It Direct Buy Address : Trident Business Park Park Neptune Way / / Rd , Huddersfield HD2 1UA Phone : 0871 971 0779 YMAR267106 YMAR267193 Measurement Measurement Notify Party TIANJIN CONTE IMPORT & EXPORT TRADING CO . , LTD B - 704-2 HAITAI BUILDING , NO.6 HUATIAN ROAD , HUAYUAN INDUSTRIAL AREA TIANJIN , CHINA FLXT - 00002617825A Seal Seal Description of Packages and Goods Description of Packages and Goods Container Containerized shipped to Consignee / FLXT00002617825A SOFA BED BED SIDEBOARD Container Tianjin Xingang Shipped From Exporter / Port of Loading Special Instructions Destination Agent Forwarding Agent NRA Number Booking Number HMM SOUTHAMPTON - 013W Places of Delivery Gross Weight Gross Weight 399310 4017189 MBL Ref . No. Port of Unloading CY - CY 70.510 cbm 68.580 cbm YMMU6077835 FCL Ltd. Direct Buy It Address : Trident Business Park / / Way / / Rd , Huddersfield HD2 1UA Phone : 0871 971 0779"},
]

In [9]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)
 
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}


Device set to use cuda:0


In [10]:
output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

Output: {'UNLADING_PORT': 'Southampton, United Kingdom', 'HTS_NUMBER': '9403.60', 'GROSS_WEIGHT': '5,540.00 kg', 'NUMBER_OF_PACKAGES': '11,056.00 kg', 'CONTAINER_NUMBER': '1854 CARTONS', 'MASTER_BILL_OF_LADING': 'YMJAN235174064', 'PLACE_OF_DELIVERY': 'Southampton, United Kingdom', 'NOTIFY_PARTY': 'Tianjin Contex Import & Export Trading Co., LTD B - 704-2 HAITAI BUILDING, NO.6 HUATIAN ROAD, HUAYUAN INDUSTRIAL AREA TIANJIN, CHINA', 'VESSEL_VOYAGE': '013W', 'PO_NUMBER': '399310 4017189', 'CONTRACT_OF_CARRIAGE': 'CY - CY', 'VOLUME': 'cbm 70.510 68.580', 'CONSIGNEE': 'FCL Ltd. Direct Buy It Address : Trident Business Park / / Way / / Rd, Huddersfield HD2 1UA Phone : 0871 971 0779'}
